In [16]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader,TensorDataset
from torchvision.datasets import FashionMNIST
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from torchvision.datasets import ImageFolder

In [17]:
!wget http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz

--2023-04-20 11:23:17--  http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108761145 (104M) [application/x-gzip]
Saving to: ‘lfw-deepfunneled.tgz.1’

lfw-deepfunneled.tg 100%[===================>] 103.72M  28.6MB/s    in 3.8s    

2023-04-20 11:23:21 (27.3 MB/s) - ‘lfw-deepfunneled.tgz.1’ saved [108761145/108761145]



In [18]:
!tar -zxvf lfw-deepfunneled.tgz

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0012.jpg
lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0013.jpg
lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0014.jpg
lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0015.jpg
lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0016.jpg
lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0017.jpg
lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0018.jpg
lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0019.jpg
lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0020.jpg
lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0021.jpg
lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0022.jpg
lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0023.jpg
lfw-deepfunneled/Luiz_Inacio

In [19]:
!mkdir './lfw-deepfunneled/train'
!mkdir './lfw-deepfunneled/test'
!mv lfw-deepfunneled/[A-W]* lfw-deepfunneled/train
!mv lfw-deepfunneled/[X-Z]* lfw-deepfunneled/test

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
mv: cannot move 'lfw-deepfunneled/Camille_Colvin' to 'lfw-deepfunneled/train/Camille_Colvin': Directory not empty
mv: cannot move 'lfw-deepfunneled/Camille_Lewis' to 'lfw-deepfunneled/train/Camille_Lewis': Directory not empty
mv: cannot move 'lfw-deepfunneled/Camryn_Manheim' to 'lfw-deepfunneled/train/Camryn_Manheim': Directory not empty
mv: cannot move 'lfw-deepfunneled/Candace_Sutton' to 'lfw-deepfunneled/train/Candace_Sutton': Directory not empty
mv: cannot move 'lfw-deepfunneled/Candice_Beatty' to 'lfw-deepfunneled/train/Candice_Beatty': Directory not empty
mv: cannot move 'lfw-deepfunneled/Candice_Bergen' to 'lfw-deepfunneled/train/Candice_Bergen': Directory not empty
mv: cannot move 'lfw-deepfunneled/Candie_Kung' to 'lfw-deepfunneled/train/Candie_Kung': Directory not empty
mv: cannot move 'lfw-deepfunneled/Carey_Lowell' to 'lfw-deepfunneled/train/Carey_Lowell': Directory not empty
mv: cannot move 'lfw-deepfunneled/Cari_Davis' to 'lfw-deepfunnel

In [20]:
# LFW 얼굴 이미지 데이터

In [21]:
# 32 x 32 픽셀의 이미지를 128 x 128픽셀로 확대 클래스
class DownSizedPairImageFoler(ImageFolder):
  def __init__(self, root, transform = None, large_size=128,samll_size=32, **kwds):
    super().__init__(root, transform=transform, **kwds)
    self.large_resizer = transforms.Resize(large_size)
    self.samll_size = transforms.Resize(samll_size)
  def __getitem__(self, index):
    path, _ =self.imgs[index]
    img = self.loader(path)
    # 읽은 이미지를 128 128 과 32 32 픽셀로 리사이즈
    large_img = self.large_resizer(img)
    small_img = self.samll_size(img)
    if self.transforms is not None:
      large_img = self.transform(large_img)
      small_img = self.transform(small_img)
    return  small_img,large_img

In [22]:
train_data = DownSizedPairImageFoler('/content/lfw-deepfunneled/train',
                                     transform = transforms.ToTensor()
                                     )
test_data = DownSizedPairImageFoler('/content/lfw-deepfunneled/test',
                                     transform = transforms.ToTensor()
                                     )


In [23]:
batch_size = 9
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(train_data,batch_size=batch_size,shuffle=False)

In [24]:
# 모델 작성
# conv2d stride = 2 로주면 폴링계층의 역활도 수행하므로 maxpool2d 넣지 않아도 이미지가 1/2 된다
# ConvTransepose2d : transposed convolution 이라는 합성곱 연산을 사용
# 합성곱 커널을 행렬로 전개했을때 원래의 합성곱 행렬의 가로 세로를 바꾸는 것(전치)
# 이것을 stride =2 이미지 크기가 약 2개가 됨

In [25]:
net = nn.Sequential(
    nn.Conv2d(3,256,4,stride=2,padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(256),

    nn.Conv2d(256,512,4,stride=2,padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(512),

    nn.ConvTranspose2d(512,256,4,stride=2,padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(256),

    nn.ConvTranspose2d(256,128,4,stride=2,padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(128),

    nn.ConvTranspose2d(128,64,4,stride=2,padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(64),

    nn.ConvTranspose2d(64,3,4,stride=2,padding=1)
)

In [26]:
# loss 함수는 원본이미지하고 확대한 이미지가 최대한 같게 측정
# 이미지나 음성신호를 복원할때는 MSE가 아니라. PNSR이라는지표를 사용
import math
def psnr(mse,max_v=1.0):
  return 10 * math.log10(max_v**2 / mse)

In [27]:
device = 'cuda'  if torch.cuda.is_available else 'cpu'
def eval_net(net, data_loader,device='gpu'):
  net.eval()
  ylist = []
  ypred = []
  for data, label in data_loader:
    data = data.to(device)
    label = label.to(device)
    with torch.no_grad():
      y_pred = net(data)
    ylist.append(label)
    ypred.append(y_pred)
  realy = torch.cat(ylist)
  redicty = torch.cat(ypred)
  # 예측 정확도 mse 계산
  return nn.functional.mse_loss(redicty,realy).item()

In [32]:
def train_net(net, train_loader, test_loader, 
              loss_fn = nn.MSELoss(),
              optimizer = optim.Adam,n_iter = 10, device='gpu'):
  op = optimizer(net.parameters())
  train_loss = []
  val_acc=[]
  for epoch in range(n_iter):
    net.train()
    n_acc = 0
    n_count = 0
    train_acc = [] 
    total_loss = 0
    for idx, (data, label) in tqdm(enumerate(train_loader)):
      data = data.to(device)
      label = label.to(device)
      y_hat = net(data)
      loss = loss_fn(y_hat,label)
      op.zero_grad()
      loss.backward()      
      op.step()
      # total_loss += loss.item()
      total_loss += float(loss)
      n_count += len(data)            
    
    train_loss.append(total_loss / len(train_loader))      
    
    #검증데이터의 예측 정확도
    val_acc.append(eval_net(net,test_loader, device))
    # epoch 의 결과를 출력
    print(epoch, train_loss[-1],psnr(train_loss[-1]),psnr(val_acc[-1]) )

In [29]:
net.to(device)

Sequential(
  (0): Conv2d(3, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): ReLU()
  (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (4): ReLU()
  (5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (7): ReLU()
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (10): ReLU()
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (13): ReLU()
  (14): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (15): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(

In [33]:
train_net(net, train_loader,test_loader,device=device)

1453it [00:56, 25.90it/s]


OutOfMemoryError: ignored

In [ ]:
from torchvision.utils import save_image
random_test_loader = DataLoader(test_data,batch_size=4, shuffle=True)
data, label =  next(iter(random_test_loader))

bl_recon =  torch.nn.functional.upsample(data,128, mode = 'bilinear', align_corners=True)
# cnn으로 확대
temp = net(data.to(device)).to('cpu')
# torch.cat으로 원본, Billiner, CNN 이미지를 결합하고
# save image로 결합한 이미지를 출력(저장)
save_image(torch.cat([label,bl_recon,temp],0), 'cnn_upscale.jpg'  ,nrow=4)

In [ ]:
from IPython.display import Image, display_jpeg
display_jpeg(Image('./cnn_upscale.jpg'))